# Esercizio 3 - Sequence tree

## Operazioni preliminari sul DS

#### Carichiamo il DS

In [1]:
import os
from datetime import datetime

def load_diabetes_dataset(verbose=False,remove_class=False)-> list: 
    folder_path="datasets\\diabetes"
    dataset = []
    errcount=0

    for filename in os.listdir(folder_path):

        file_path = os.path.join(folder_path, filename)

        if os.path.isfile(file_path)and filename.startswith('data'):
            entry=[]

            with open(file_path, 'r') as file:
                content = file.readlines()
                for line in content:
                    item = tuple((line[0:-1] if line.endswith('\n') else tuple(line)).split("\t"))

                    # If the item is valid, append it to the entry
                    try:
                        item_f = datetime.strptime(item[0]+" "+item[1], "%m-%d-%Y %H:%M")
                        if (not remove_class):
                            entry.append((item_f,item[2],item[3]))
                        elif (remove_class and item[2]!="65"):
                            entry.append((item_f,item[2],item[3]))
                    except:
                        if(verbose):
                            print(f"[!] Entry {item} in file {filename} is NOT vallid. Skipped!")
                        else:
                            errcount+=1
                # add the entry to the dataset
                dataset.append(entry)
    print(f"Skipped {errcount} items.")
    return dataset


#### A partire dal dataset grezzo generiamo X,Y,ST e V.

> Definiamo la classe reale come uno [0,1] che indica se l'evento 65 (65 = Hypoglycemic symptoms) si è verificato in una certa finestra di tempo `event_window` dopo un certo waiting time `waiting_window`.

In [2]:
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta


#Domande. 
#2.  Also, la distanza d nelle coppie d,l è definita in giorni?

dataset_X = load_diabetes_dataset()

def compute_datasets(dataset:list,observation_window,waiting_window,prediction_window)->list:
    # Initialize: I assume it will not be found
    dataset_X = [0] * len(dataset)
    dataset_Y = [0] * len(dataset)

    # each entry
    for i in range(0,len(dataset)):
        entry = dataset[i]
        #i need to isolate observation items.
        j=0

        end_observation = entry[0][0] + observation_window # this is the timestamp of the first item in the entry
        curr_time = entry[0][0]

        observation_items = []

        #1. Isolate observation window! That's our actual X dataset!
        while (curr_time <= end_observation):
            #print(f"{curr_time}  <=  {end_observation}")
            if (entry[j][1]!="65"):
                observation_items.append(entry[j])
            j+=1
            curr_time = entry[j][0]
        dataset_X[i] = observation_items 

        #2. Skip the waiting window
        end_waiting =  entry[0][0] + observation_window + waiting_window

        while(curr_time <= end_waiting):
            j+=1
            curr_time = entry[j][0]

        #3. Generate prediction window and assign 1 in Y if adverse effect is present, 0 else

        while (curr_time <= entry[0][0]+observation_window+waiting_window+prediction_window):
            if (entry[j][1]=="65"):
                dataset_Y[i] = 1
                break
            j+=1
            if (j<len(entry)):
                curr_time = entry[j][0]
            else:
                break
    
    dataset_ST = [0]*len(dataset_X)
    for i in range(0,len(dataset_X)):
        dataset_ST[i] = dataset_X[i][0][0]
    dataset_V = [None]*len(dataset_X)

    return dataset_X,dataset_Y,dataset_ST,dataset_V



observation_window = relativedelta(days=+7)
waiting_window = relativedelta(days=+0)
prediction_window = relativedelta(days=+5)

dataset_X,dataset_Y, dataset_ST,dataset_V = compute_datasets(dataset_X,observation_window,waiting_window,prediction_window)

Skipped 46 items.


#### Stampiamo il DS elaborato.

In [3]:
def print_dataset_state(dataset_X,dataset_Y,dataset_ST,dataset_V,howmany=10):
    print("X\t\tY\tST\t\t\tV")
    for i in range(0,len(dataset_X[:howmany])):
        print(f"len:{len(dataset_X[i])}\t\t{dataset_Y[i]}\t{dataset_ST[i]}\t{dataset_V[i]}")

print_dataset_state(dataset_X,dataset_Y,dataset_ST, dataset_V,len(dataset_X))
print(f"# entries: {len(dataset_X)}")

X		Y	ST			V
len:46		1	1991-04-21 09:09:00	None
len:56		0	1989-10-10 08:00:00	None
len:64		1	1990-07-21 06:43:00	None
len:55		0	1990-08-19 17:00:00	None
len:61		0	1990-09-01 16:48:00	None
len:51		0	1989-04-29 08:00:00	None
len:54		0	1989-03-27 22:00:00	None
len:33		1	1990-07-31 12:09:00	None
len:57		0	1990-04-22 18:08:00	None
len:51		0	1989-02-18 08:00:00	None
len:55		1	1990-07-13 09:44:00	None
len:68		1	1990-07-22 09:53:00	None
len:76		1	1990-09-04 05:53:00	None
len:51		1	1991-03-11 18:15:00	None
len:33		1	1991-04-13 08:47:00	None
len:31		1	1991-05-22 07:24:00	None
len:58		1	1990-07-13 09:48:00	None
len:63		1	1990-08-18 07:16:00	None
len:60		1	1990-09-09 17:23:00	None
len:50		0	1991-05-12 06:55:00	None
len:65		0	1989-09-03 08:00:00	None
len:55		0	1991-03-14 22:05:00	None
len:51		0	1991-04-27 23:02:00	None
len:37		0	1991-05-28 21:35:00	None
len:19		1	1990-07-24 16:00:00	None
len:37		0	1988-07-13 08:00:00	None
len:16		0	1989-01-29 08:00:00	None
len:27		0	1989-11-05 07:00:00	None
len:59		

In [7]:
import itertools
import random
import math
random.seed(1)


# Find al possible d,l couples that I could split the tree on. Note: Ds are randomly selected bc otherwise I'd end up with ~36000 pairs...
def create_pairs(dataset_X:list,dataset_ST:list,all_d=False,howmany_d =30):
    #1. Create all labels available from current.
    labels = set()
    for i in range(0,len(dataset_X)):
        for item in dataset_X[i]:
            if (item[0] > dataset_ST[i]):# Only consider label if it's not been superato
                labels.add(item[1])

    durations = set()

    #2. Find all d
    for i in range(0,len(dataset_X)):
        for item in dataset_X[i]:
            if (item[0] > dataset_ST[i]): # Only consider timestamp if it's not been superato
                durations.add(item[0]-dataset_ST[i])
    durations = random.sample(sorted(durations),howmany_d) # Is this ok?
    return list(itertools.product(durations,labels))

# Given a set with binary classes, computes entropy
def compute_entropy(dataset_Y):
    #print(dataset_Y)
    ones = len(list(filter(lambda classification : classification == 1,dataset_Y)))
    zeros = len(list(filter(lambda classification : classification == 0,dataset_Y)))

    if(ones == 0 or zeros==0):
        return 0
    #print(f"count 1: {ones}") #print(f"count 0: {zeros}")
    #print(f"p:{ones/len(dataset_X)}  1-p:{zeros/len(dataset_X)}")
    entropy = ones/len(dataset_Y)*math.log2(1/(ones/len(dataset_Y))) + zeros/len(dataset_Y)*math.log2(1/(zeros/len(dataset_Y)))

    return entropy


# Given a pair of duration d and label l, computes its information gain on the dataset if we were to split it according to the sequence tree rules.
def compute_IG(dl_pair,dataset_X,dataset_Y,verbose=False):
    if verbose:
        print(f"Computing IG for {dl_pair}")
    indexes_t = [] # indexes of entries that have label==l within d time
    indexes_f = [] # indexes of entries that have DON'T HAVE label==l within d time
    d,l = dl_pair

    entropy_0 = compute_entropy(dataset_Y)
    #print(f"Initial entropy: {entropy_0}")

    #1. Separate entries that satisfy event test from those who don't
    for i in range(0,len(dataset_X)):
        entry = dataset_X[i]
        found=False

        for item in entry:
            #starting from the starting time, see if it exists an item with timestamp < d and label == l
            if(item[0]> dataset_ST[i]): # If i'm over starting time
                if(item[0]< dataset_ST[i]+d and item[2]==l):
                    found=True
        if(found):
            indexes_t.append(i)
        else:
            indexes_f.append(i)
    
    #2. Compute final entropy. first let's generate our new datasets...
    dataset_Yt=[ dataset_Y[i] for i in indexes_t]
    dataset_Yf=[ dataset_Y[i] for i in indexes_f]
    #print(dataset_Yt)    print(dataset_Yf)
            
    entropy_f = (len(indexes_t)/len(dataset_X))*compute_entropy(dataset_Yt) + (len(indexes_f)/len(dataset_X))*compute_entropy(dataset_Yf)

    if verbose:
        print(indexes_t,indexes_f)
        print(dataset_Yt,dataset_Yf)
        #print(f"final entropy: {entropy_f}")
        print(f"information gain is {entropy_0-entropy_f}")

    return(entropy_0-entropy_f)

# Given all possible pairs of d,l finds the one with the highest information gain (aka the one I should actually split on)
def max_IG(dataset_X,dataset_Y, dataset_ST, verbose=False):
    dl_pairs = create_pairs(dataset_X,dataset_ST)
    #igs_list=[(x,compute_IG(x,dataset_X,dataset_Y)) for x in dl_pairs[:] ]
    igs_dict = {x:compute_IG(x,dataset_X,dataset_Y) for x in dl_pairs[:10]}
    if verbose:
        print(igs_dict)
        print(f"Max IG is {igs_dict[max(igs_dict, key=igs_dict.get)]} for entry {max(igs_dict, key=igs_dict.get)}")
    return (max(igs_dict, key=igs_dict.get),igs_dict[max(igs_dict, key=igs_dict.get)])

print(max_IG(dataset_X,dataset_Y,dataset_ST))


((datetime.timedelta(days=1, seconds=2880), '35'), 0.04185721895377048)


In [10]:
from treelib import Node, Tree
class SequenceTree(Tree):
    def __init__(self, tree=None, deep=False, node_class=None, identifier=None):
        super(SequenceTree, self).__init__(tree=tree, deep=deep, node_class=node_class, identifier=identifier)

    def create_node(self, tag=None, identifier=None, parent=None, data=None,branch=None):
        """
        Create a child node for the given @parent node. If ``identifier`` is absent,
        a UUID will be generated automatically.
        """
        new_node = super(SequenceTree, self).create_node(tag=tag, identifier=identifier, parent=parent, data=data)
        
        siblings = super(SequenceTree,self).siblings(new_node.identifier)
        

        if len(super(SequenceTree,self).siblings(new_node.identifier))>=2:
           raise ValueError("Parent node already has maximum number of children")
        if branch in [x.data[0] for x in siblings]:
           raise ValueError(f"Parent node already has a {branch} branch")
        return new_node
    
    def display(self):
        print(self.show(stdout=False))
    
    def create_node_event(self,data,parent=None,branch=None):
        branch_f = "" if (branch is None) else branch+" "
        tag =  "\x1b[32m⬤\x1b[0m" +" "+ branch_f +"[" + str(data[1])+", " + format_timedelta(data[0])+ "] "  

        return     self.create_node(tag,data=(branch,{"classification":data}),parent=parent,branch=branch)

def format_timedelta(td: timedelta) -> str:
    days = td.days
    hours, remainder = divmod(td.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    
    formatted_str = ""
    if days:
        formatted_str += f"{days}d"
    if hours:
        formatted_str += f"{hours}h"
    if minutes:
        formatted_str += f"{minutes}m"
    if seconds:
        formatted_str += f"{seconds}s"
    
    return formatted_str if formatted_str else "0s"

def create_node_event(tree,data,parent=None,branch=None):
    branch_f = "" if (branch is None) else branch+" "
    tag =  "\x1b[32m⬤\x1b[0m" +" "+ branch_f +"[" + str(data[1])+", " + format_timedelta(data[0])+ "] "  

    return     tree.create_node(tag,data=(branch,{"classification":data}),parent=parent,branch=branch)

def create_node_class(tree,classification,parent=None,branch=None):
    branch_f = "" if (branch is None) else branch+" "
    tag =  "\x1b[33m◆\x1b[0m" +" "+ branch_f + " " + str(classification)  

    return     tree.create_node(tag,data=(branch,{"classification":classification}),parent=parent,branch=branch)

def create_node_value(tree,data,parent=None,branch=None):
    branch_f = "" if (branch is None) else branch+" "
    tag =  "\x1b[31m■\x1b[0m" +" "+ branch_f +"[" + str(data[1])+", " + format_timedelta(data[0])+ "] "  

    print(data)
    return     tree.create_node(tag,data=(branch,{"value":data}),parent=parent,branch=branch)


# Example tree:
tree = SequenceTree()

# Each node has:
# - Tag (string to be printed in the tree)
# - Data (the actual info) is a tuple with:
#  - branch --> "t" or "f"
#  - A dict, which changes according to the type of node:
#     . event: {"event":(d,l)}
#     . value: {"value":v}
#     . classification (leaf): {"classification":class}
max_dl,ig=max_IG(dataset_X,dataset_Y,dataset_ST)
print(max_dl)

root = create_node_event(tree,max_dl)
#a = create_node_event(tree,max_dl,parent=root.identifier,branch="t")
#b = create_node_value(tree,max_dl,parent=root.identifier,branch="f")
#c = create_node_event(tree,max_dl,parent=b,branch="f")



print(tree.show(stdout=False))

(datetime.timedelta(days=2, seconds=32220), '35')
⬤ [35, 2d8h57m] 



## let's start

In [ ]:
ds = load_diabetes_dataset()

dataset_X,dataset_Y,dataset_ST, dataset_V = compute_datasets(ds,)